In [38]:
## Ogni .csv è strutturato in questo modo:
    # n x (12 + 4)
    # [f1, f2, ..., f12, l0, l1, l2, l3]
# dove f_i sono le features, mentre li sono le labels; n è il numero di righe, cioè di samples.
    # l0 = BASO LEFT 2 LIVELLI
    # l1 = BASO LEFT 4 LIVELLI
    # l2 = BASO RIGHT 2 LIVELLI
    # l3 = BASO RIGHT 4 LIVELLI

import csv
import statistics
from statistics import mode 
import pandas as pd
import os
import datetime
import time    # Per capire quanto tempo ci mette 
import progressbar

sliding = True
sliding_pace = 1
predict_central_sample = False

discard_transitions = True # Devo eliminare le finestre che contengono le transizioni
discarded_windows = 0
th = 1

window_size = 200
features_number = 10 # Tutto tranne il basografico
label = 0    # Label da selezionare -> Se il .csv ne contiene una sola, label = 0
label_count = 1    # Questa variabile serve per fargli capire fino a che riga deve prendere le features
                   # Deve essere uguale al numero di labels presenti nel file .csv originale
                   # Se ne hai 4 totali, allora devi eliminare dal computo delle features 4 labels
label_column = 10    # Colonna da cui estrarre la feature - serve per calcolare il primo swing
# row_count = 486600

start = 1    # Subject from which the count starts
end = 26    # Subject at which the count ends 

strides_limit = 30  #If set to -1 no limit, otherwise stop after processing N strides

floating_point_precision = 3

in_path = "../subjects/min-max/"
#out_path = "../subjects/min-max/windows_" + str(window_size) +"/tr-" + str(discard_transitions) + "_sliding_" + str(sliding_pace) + "_c-" + str(predict_central_sample) + "/"
out_path = "../subjects/min-max/windows_" + str(window_size) + "-d-" + str(floating_point_precision) + "-tr-" + str(discard_transitions) + "-sl-" + str(sliding)
if sliding:
    out_path += "-pace-" + str(sliding_pace) 
out_path += "/"
out_path_discarded = out_path + "discarded/"
prefix = 'Biphase_Subject'
suffix = '_norm'
out_suffix = '_norm_windows_'

#create folders if not existing...
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path_discarded):
    os.makedirs(out_path_discarded)

for subj in range(start, end + 1):
    
    current_phase = -1
    tot_phases = 0
    
    start_time = time.time()
    #count rows in file
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        row_count = sum(1 for row in csv_reader)
        
    ## -- To find the first Swing
    # Serve per far partire tutti i soggetti (e le finestre) dal primo swing registrato
    csv_swing = []
    found = False    # Flag per stoppare il ciclo quando trova il primo Swing
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        swing_onset = 0
        for line in csv_reader: 
            if not found:
                csv_swing.append(line)
                swing_onset += 1
                if line[label_column] == '1.000000':
                    found = True
    print('Swing Onset at sample ' + str(swing_onset))
    
    csv_in = []
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for line in csv_reader: 
            csv_in.append(line)
    
    with open(out_path + prefix + str(subj) + out_suffix + str(window_size) + '.csv', mode='w') as csv_out_file:
        with open(out_path_discarded + prefix + str(subj) + out_suffix + str(window_size) + '.txt', mode='w') as discarded_file:
            csv_writer = csv.writer(csv_out_file, delimiter=',')

            print("Subject " + str(subj) + " rows: " + str(row_count))

            discarded_windows = 0

            if sliding:
                pace = sliding_pace    
                limit = row_count-window_size    # Questo serve per non superare i limiti del for 
            else:
                pace = window_size
                limit = row_count-window_size
                
            for k in progressbar.progressbar(range(swing_onset - 1, limit, pace)):    # Qui tira fuori la progressbar, per renderti conto di come cicla e di quanto tempo ci mette
                #time.sleep(0.0001)    # Troppo grande rallenta il for!
                window_rows = []
                for i in range(k, k + window_size):
                    window_rows.append(csv_in[i])
                window_rows = pd.DataFrame(window_rows).astype(float)
                bsx = window_rows[features_number + label]    
                if predict_central_sample:
                    m = bsx[int(len(bsx)/2)]
                else:
                    m = bsx.mode()[0]
                    
                if current_phase == -1:
                    current_phase = m
                if m != current_phase:
                    tot_phases += 1
                    current_phase = m
                    print("Phases: " + str(tot_phases))
                    if strides_limit != -1 and tot_phases/2 >= strides_limit:
                        break
                
                occ = len(window_rows[window_rows[features_number + label] == float(m)])    # Occorrenza della label nella finestra
               
                if not discard_transitions or occ/window_size >= th:       # Qua dobbiamo capire come scartarle le finestre (cioè che threshold usare!)
                    single_row_window = window_rows.iloc[:, :-label_count].values.reshape(1,-label_count).flatten().tolist()
                    single_row_window = [ round(elem, floating_point_precision) for elem in single_row_window ]
                    single_row_window.append(int(m))
                    csv_writer.writerow(single_row_window)
                else:
                    discarded_windows += 1
                    discarded_file.write("Window starting at row " + str(k) + "\n")
                    discarded_file.write(str(window_rows) + "\n\n\n")
                    #print("Discarded windows: " + str(discarded_windows))
            end_time = time.time()        
            time_tot = int(end_time - start_time)
            print('Subject ' + str(subj) + ' windowed. Time elapsed: ' + str(datetime.timedelta(seconds=time_tot)) + '\n')

Swing Onset at sample 6348


  0% (25 of 480053) |                    | Elapsed Time: 0:00:00 ETA:   0:32:38

Subject 1 rows: 486600


  0% (1078 of 480053) |                  | Elapsed Time: 0:00:05 ETA:   1:19:25

Phases: 1


...

Phases: 58


 23% (62883 of 271678) |###              | Elapsed Time: 0:06:52 ETA:   0:37:24

Phases: 59


 23% (64113 of 271678) |####             | Elapsed Time: 0:07:00 ETA:   0:33:06

Phases: 60
Subject 26 windowed. Time elapsed: 0:07:02



In [7]:
swing_on = 33
row = 204
windows_size = 20

actual = row - swing_on
actual
delta = actual%windows_size

actual-delta

160